In [ ]:
# 各セクターごとに全期間にわたって存在する銘柄を特定する
# 各セクターで利用可能な全銘柄を取得
all_stocks_per_sector = {sector: df[df['セクター'] == sector]['銘柄'].unique() for sector in sectors}

# 全期間にわたってデータが存在する銘柄を特定
available_stocks_per_sector = {}
for sector, stocks in all_stocks_per_sector.items():
    available_stocks = []
    for stock in stocks:
        if len(df[(df['セクター'] == sector) & (df['銘柄'] == stock)]) == len(dates):
            available_stocks.append(stock)
    available_stocks_per_sector[sector] = available_stocks

# 各セクターごとに発行額の最大値、中央値、最小値を持つ銘柄を選定
selected_stocks = {}
for sector, stocks in available_stocks_per_sector.items():
    sector_df = df[(df['セクター'] == sector) & (df['銘柄'].isin(stocks))]
    
    max_stock = sector_df.loc[sector_df['発行額'].idxmax()]['銘柄']
    min_stock = sector_df.loc[sector_df['発行額'].idxmin()]['銘柄']
    
    # 中央値に最も近い値を持つ銘柄を特定
    median_val = sector_df['発行額'].median()
    closest_median_stock = (sector_df.iloc[(sector_df['発行額'] - median_val).abs().argsort()[:1]])['銘柄'].values[0]
    
    selected_stocks[sector] = [max_stock, closest_median_stock, min_stock]

# PDFファイルの作成
with PdfPages(pdf_filename) as pdf:
    for sector in sectors:
        for stock in selected_stocks[sector]:
            stock_df = df[(df['セクター'] == sector) & (df['銘柄'] == stock)]
            plt.figure(figsize=(10, 6))
            sns.lineplot(x='Date', y='リターン', data=stock_df)
            plt.title(f'{sector} Sector - {stock} Return Trend')
            plt.xlabel('Date')
            plt.ylabel('Return')
            pdf.savefig()
            plt.close()



In [ ]:
# PDFファイルの作成
pdf_filename = "/mnt/data/sector_wise_stocks_return_trends.pdf"
with PdfPages(pdf_filename) as pdf:
    for sector in sectors:
        sector_df = df[df['セクター'] == sector]

        # 最大値、最小値、中央値に最も近い値を持つ異なる銘柄を選定
        max_stock = sector_df.loc[sector_df['発行額'].idxmax()]['銘柄']
        min_stock = sector_df[sector_df['銘柄'] != max_stock].loc[sector_df['発行額'].idxmin()]['銘柄']
        median_val = sector_df['発行額'].median()
        closest_median_stock = sector_df[~sector_df['銘柄'].isin([max_stock, min_stock])].iloc[(sector_df['発行額'] - median_val).abs().argsort()[:1]]['銘柄'].values[0]
        
        # グラフの描画
        plt.figure(figsize=(10, 6))
        sns.lineplot(x='Date', y='リターン', data=sector_df[sector_df['銘柄'] == max_stock], label=f'{max_stock} (Max)')
        sns.lineplot(x='Date', y='リターン', data=sector_df[sector_df['銘柄'] == closest_median_stock], label=f'{closest_median_stock} (Median)')
        sns.lineplot(x='Date', y='リターン', data=sector_df[sector_df['銘柄'] == min_stock], label=f'{min_stock} (Min)')

        plt.title(f'{sector} Sector Return Trend')
        plt.xlabel('Date')
        plt.ylabel('Return')
        plt.legend()
        pdf.savefig()
        plt.close()

pdf_filename




In [ ]:
# データセットからすべてのユニークなセクターを抽出して定義
sectors = df['セクター'].unique()

# 各セクターで利用可能な全期間にわたって存在する銘柄を特定する
available_stocks_per_sector = {}
for sector in sectors:
    available_stocks = []
    sector_df = df[df['セクター'] == sector]

    for stock in sector_df['銘柄'].unique():
        stock_dates = sector_df[sector_df['銘柄'] == stock]['Date'].unique()
        if len(stock_dates) == len(dates):
            available_stocks.append(stock)

    available_stocks_per_sector[sector] = available_stocks

available_stocks_per_sector



In [ ]:
import pandas as pd

# 辞書データを仮定（実際のモデル結果に基づいて調整してください）
# ここでのdict_は、すでに上記の形式でデータを持っていると仮定しています

# Excelファイルへの出力
with pd.ExcelWriter('regression_results_by_sector.xlsx') as writer:
    # 各セクターと時点の組み合わせに対してループ
    for (month, sector), results in dict_.items():
        # パラメータ名を取得
        param_names = results["parameters"].index.tolist()
        
        # DataFrameを作成
        data = {
            "R-Square": [results["R-Square"]],
            **{f"Param_{param}": [results["parameters"][param]] for param in param_names},
            **{f"T-Value_{param}": [results["t-value"][param]] for param in param_names},
            **{f"P-Value_{param}": [results["p-value"][param]] for param in param_names},
        }
        df = pd.DataFrame(data, index=[month])
        
        # 既にシートが存在する場合は、そのシートにデータを追加
        if sector in writer.sheets:
            old_df = pd.read_excel(writer.path, sheet_name=sector, index_col=0)
            df = pd.concat([old_df, df])
            
        # シートにDataFrameを書き込み
        df.to_excel(writer, sheet_name=sector)

# このコードは、指定された形式で各セクターごとに異なるシートに結果を出力します。
# 各シートでは、月をインデックスとして、R-Square、各パラメータの値、t値、p値がカラムになります。
